### Dataset: Brazilian E-Commerce Public Dataset by Olist

Conjunto de dados públicos de comércio eletrônico brasileiro por Olist

Este é um conjunto de dados públicos de comércio eletrônico brasileiro de pedidos feitos na Olist Store. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Seus recursos permitem visualizar um pedido em múltiplas dimensões: desde status do pedido, preço, desempenho de pagamento e frete até localização do cliente, atributos do produto e finalmente avaliações escritas pelos clientes. Também um conjunto de dados de geolocalização que relaciona os CEPs brasileiros às coordenadas lat/lng.

Estes são dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.

Olist, a maior loja de departamentos dos mercados brasileiros. Olist conecta pequenas empresas de todo o Brasil a canais sem complicações e com um único contrato. Esses comerciantes podem vender seus produtos através da Olist Store e enviá-los diretamente aos clientes usando os parceiros logísticos da Olist. Veja mais em nosso site: www.olist.com

Depois que um cliente compra o produto na Olist Store, um vendedor é notificado para atender ao pedido. Assim que o cliente recebe o produto, ou vence a data estimada de entrega, o cliente recebe por e-mail uma pesquisa de satisfação onde pode dar uma nota da experiência de compra e anotar alguns comentários.
Atenção

Um pedido pode ter vários itens.
Cada item pode ser entregue por um vendedor distinto.
Todos os textos de identificação de lojas e parceiros foram substituídos pelos nomes das grandes casas de Game of Thrones.


PNL:

Este conjunto de dados oferece um ambiente supremo para analisar o texto das avaliações em suas múltiplas dimensões.

Agrupamento:

Alguns clientes não escreveram um comentário. Mas por que eles estão felizes ou bravos?

Previsão de vendas:

Com as informações da data de compra, você poderá prever vendas futuras.

Desempenho de entrega:

Você também poderá trabalhar o desempenho da entrega e encontrar maneiras de otimizar os prazos de entrega.

Qualidade do produto:

Divirta-se descobrindo as categorias de produtos mais propensas à insatisfação do cliente.

Engenharia de recursos:

Crie recursos a partir deste rico conjunto de dados ou anexe algumas informações públicas externas a ele.

# Importando bibliotecas

In [114]:
%%capture

!python -m spacy download pt_core_web_md

In [138]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import requests
from tqdm import tqdm
from sklearn.model_selection import (train_test_split as tst,
                                     RandomizedSearchCV as rcv)   
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
                               
import os

# NLP MODULES
import string
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentAnalyzer
import spacy
import nltk
from spacy.lang.pt.stop_words import STOP_WORDS as pt_stop


from dotenv import (find_dotenv,
                    load_dotenv)

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
nlp=spacy.load("pt_core_news_md")
nltk.download("punkt")
nltk.download("stopwords")
stop_words=nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chenyenpin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenyenpin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [144]:
len(stop_words)

207

In [145]:
",".join(stop_words)
not_stop=["era","eram","éramos","esta","está","estamos","estão","estar",
          "estava","estavam","estávamos","esteja","estejam","estejamos","esteve","estive","estivemos",
          "estiver","estivera","estiveram","estivéramos","estiverem","estivermos","estivesse","estivessem",
          "estivéssemos","estou"]


'a,à,ao,aos,aquela,aquelas,aquele,aqueles,aquilo,as,às,até,com,como,da,das,de,dela,delas,dele,deles,depois,do,dos,e,é,ela,elas,ele,eles,em,entre,era,eram,éramos,essa,essas,esse,esses,esta,está,estamos,estão,estar,estas,estava,estavam,estávamos,este,esteja,estejam,estejamos,estes,esteve,estive,estivemos,estiver,estivera,estiveram,estivéramos,estiverem,estivermos,estivesse,estivessem,estivéssemos,estou,eu,foi,fomos,for,fora,foram,fôramos,forem,formos,fosse,fossem,fôssemos,fui,há,haja,hajam,hajamos,hão,havemos,haver,hei,houve,houvemos,houver,houvera,houverá,houveram,houvéramos,houverão,houverei,houverem,houveremos,houveria,houveriam,houveríamos,houvermos,houvesse,houvessem,houvéssemos,isso,isto,já,lhe,lhes,mais,mas,me,mesmo,meu,meus,minha,minhas,muito,na,não,nas,nem,no,nos,nós,nossa,nossas,nosso,nossos,num,numa,o,os,ou,para,pela,pelas,pelo,pelos,por,qual,quando,que,quem,são,se,seja,sejam,sejamos,sem,ser,será,serão,serei,seremos,seria,seriam,seríamos,seu,seus,só,somos,sou,sua,suas,também,t

# Funcoes necessarias para desenvolvimento do trabalho

In [125]:
# Function to get the datas
def load_data(path:str, path1:str, path2:str)->pd.DataFrame:
    '''
    params:
    path:string
    return dataframe
    '''
    load_dotenv(find_dotenv())
    path=os.getenv("path")
    path1=os.getenv("path1") 
    path2=os.getenv("path2")
    df_rev=pd.read_csv(path)
    def_cust=pd.read_csv(path1)
    df_item=pd.read_csv(path2)
    return df_rev, def_cust, df_item

#--------------------------------------------------------------
# Eda Function
def eda(df:pd.DataFrame)->None:
  '''
  Retornara um analise exploratoria simples

  -param df: pandas dataframe
  '''
  print("-"*30,"DataFrame Shape", "-"*30)
  print(f"Columns {df.shape[0]}\nRows {df.shape[1]}")
  print("-"*30,"DataFrame info", "-"*30)
  print(df.info())
  print("-"*30,"DataFrame Nan", "-"*30)
  print(df.isna().sum())
  print("-"*30,"DataFrame duplicated", "-"*30)
  print(df.duplicated().sum())
  print("-"*30,"DataFrame nunique", "-"*30)
  print(df.nunique())
  
#---------------------------------------------------------------------

#Lower casing the words

def lower_case(column):
   '''
   Retorna as coluna selecionada para lower case
   '''
   return column.lower()

#---------------------------------------------------------
def remove_punctuation(column):
    '''
    Function to remove punctuation from the input string
    '''
    column = re.sub(f"[{re.escape(string.punctuation)}]", "", column)
    return column

#-------------------------------------------------------

def tokenize(columns:str)->pd.Series:
   tokenize= word_tokenize(columns)
   column_withno_stopwords=[token for token in tokenize  if not token in stop_words ]
   new_sentence=" ".join(column_withno_stopwords)
   return new_sentence


In [117]:
df_rev, df_cust, df_item=load_data("path","path1","path2")
df_item=df_item[['order_id',
     'price', 'freight_value']]
df_rev=df_rev[['order_id', 'review_score',
       'review_comment_message', 'review_creation_date']]

In [118]:
df=pd.merge(df_rev, df_item, how="inner", on="order_id").copy()

In [119]:
df.dropna(inplace=True)

In [120]:
df=df.reset_index(drop=True)

In [130]:
df['review_comment_message'] = df['review_comment_message'].apply(lambda x :lower_case(x))
df["review_comment_message"]=df["review_comment_message"].apply(lambda x :remove_punctuation(x))
df["token"]=df["review_comment_message"].apply(lambda x: tokenize(x))

In [132]:
df["review_comment_message"]

0                     recebi bem antes do prazo estipulado
1        parabéns lojas lannister adorei comprar pela i...
2        aparelho eficiente no site a marca do aparelho...
3                 mas um pouco travandopelo valor ta boa\n
4        vendedor confiável produto ok e entrega antes ...
                               ...                        
47637    para este produto recebi de acordo com a compr...
47638    entregou dentro do prazo o produto chegou em c...
47639    o produto não foi enviado com nf não existe ve...
47640    excelente mochila entrega super rápida super r...
47641    meu produto chegou e ja tenho que devolver poi...
Name: review_comment_message, Length: 47642, dtype: object